### Boot Finder 

In [1]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

Checking and installing required python dependencies in requirements.txt
Done


In [2]:
# Import necessary libraries
import os

import coremltools 
from skafossdk import *
import turicreate as tc
import pandas as pd
from s3fs.core import S3FileSystem

In [3]:
# Initialize Skafos
ska = Skafos()

In [4]:
#Pull in boot images from S3 
_local_dir = 'BootImages'

if not os.path.exists(_local_dir):
    os.makedirs(_local_dir)

# Connect to S3
s3 = S3FileSystem(anon=True)

# Bring in boot data
boots = s3.ls("s3://mf.testing.bucket/Boots/Images/")
        
# Loop over the image paths, adding them to the SFrames
for b in boots: 
    _local_file = "/".join(b.split("/")[-1:])
    _local_path = _local_dir + "/" + _local_file
    print(_local_path)
    s3.get("s3://" + b, _local_path) # download the image

BootImages/IMG_3434.JPG
BootImages/IMG_3435.JPG
BootImages/black-uggs1.jpg
BootImages/black-uggs2.jpg
BootImages/black-uggs3.jpg
BootImages/dansko-brown1.jpg
BootImages/dansko-brown2.jpg
BootImages/dansko-brown3.jpg
BootImages/frye-brown-canvas1.jpg
BootImages/frye-brown-canvas2.jpg
BootImages/frye-brown-canvas3.jpg
BootImages/nwob-plume1.jpg
BootImages/nwob-plume2.jpg
BootImages/nwob-plume3.jpg


In [5]:
# Pull in boot metadata
csv = "s3://mf.testing.bucket/Boots/BootMetadata_20190302.csv"
print("/".join(csv.split("/")[-1:]))
s3.get(csv, "/".join(csv.split("/")[-1:]))

BootMetadata_20190302.csv


In [6]:
# Create SFrames
metadata = tc.SFrame.read_csv("BootMetadata_20190302.csv")
boot_data  = tc.image_analysis.load_images('BootImages')

Finished parsing file /home/jovyan/BootMetadata_20190302.csv

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Parsing completed. Parsed 14 lines in 0.026895 secs.

Finished parsing file /home/jovyan/BootMetadata_20190302.csv

Parsing completed. Parsed 14 lines in 0.02662 secs.

In [7]:
#Join create image_name to merge SFrames
boot_data['Image Name'] = boot_data['path'].apply(lambda x: x.split("/")[-2:][1])
boot_data = boot_data.join(metadata, on="Image Name", how="left")
boot_data = boot_data.add_row_number()

In [8]:
boot_data.print_rows(num_rows=14)

+----+-------------------------------+--------------------------+
| id |              path             |          image           |
+----+-------------------------------+--------------------------+
| 0  |    BootImages/IMG_3434.JPG    | Height: 3024 Width: 4032 |
| 1  |    BootImages/IMG_3435.JPG    | Height: 3024 Width: 4032 |
| 2  |   BootImages/black-uggs1.jpg  | Height: 1600 Width: 1376 |
| 3  |   BootImages/black-uggs2.jpg  | Height: 1242 Width: 1600 |
| 4  |   BootImages/black-uggs3.jpg  | Height: 1214 Width: 1600 |
| 5  |  BootImages/dansko-brown1.jpg | Height: 1517 Width: 1126 |
| 6  |  BootImages/dansko-brown2.jpg | Height: 1590 Width: 931  |
| 7  |  BootImages/dansko-brown3.jpg | Height: 1560 Width: 863  |
| 8  | BootImages/frye-brown-canv... | Height: 1600 Width: 1200 |
| 9  | BootImages/frye-brown-canv... | Height: 1200 Width: 1600 |
| 10 | BootImages/frye-brown-canv... | Height: 1600 Width: 1200 |
| 11 |   BootImages/nwob-plume1.jpg  | Height: 1600 Width: 1200 |
| 12 |   B

In [13]:
model = tc.image_similarity.create(boot_data)

Performing feature extraction on resized images...
Completed 14/14


Starting brute force nearest neighbors model training.

In [14]:
img = tc.Image("IMG_3436.JPG")
model.query(img)

Performing feature extraction on resized images...
Completed 1/1


Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 7.14286     | 521us        |

| Done         |         | 100         | 2.431ms      |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,4,20.709052062663655,1
0,5,21.351998834623366,2
0,6,21.739325560663943,3
0,0,22.293770252521295,4
0,13,22.751146944450532,5


In [15]:
# Specify the CoreML model name
model_name = 'ImageSimilarity'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Use coremltools to convert model weights to half-precision.
# This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

2019-03-02 17:19:41,795 - skafossdk.asset_manager - INFO - Connecting to AssetEngine
2019-03-02 17:19:41,959 - skafossdk.asset_manager - INFO - AssetManager Connection Opened


<Request at 0x7f6c201e9c50 request_id=0f187784-b0f7-41df-990d-4cd397bbfce5 status=running time=2.3009410559898242>

asset_name: ImageSimilarity
normalized_name: imagesimilarity
filename: imagesimilarity-1551547194256.zip
version: 1551547194256
tags: ['latest']
project_token: b651d71134e086a2f7172a38
deployment_id: cc13d077-d454-4b6e-a607-878a1a351874
job_id: 
access: public
hash: 
inserted_at: 2019-03-02 17:19:58.399
updated_at: 2019-03-02 17:19:58.399
metadata: 


In [16]:
# Deliver asset to devices (push)
ska.asset_manager.deliver(
  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
  tag='latest',                   # User-defined tags to help distinguish your asset.
  dev=True                        # Push asset through Apple's APNS dev or prod server
)

<Request at 0x7f6c201d6048 request_id=3dbafaa1-fec1-4f5f-868e-d7feb1ae5163 status=running time=0.00597566700889729>

2019-03-02 17:20:05,837 - skafossdk.asset_manager - INFO - request_id: "3dbafaa1-fec1-4f5f-868e-d7feb1ae5163"
ref_type: "result"
result: "{\"io.skafos.BootFinder\":{\"success\":\"Successfully pushed 682\"}}"

